## Train/test split 

In [2]:
import argparse
import os 
import glob
import numpy as np
from transforms3d.quaternions import mat2quat

origin_dir = '../data/7Scenes/'
datasets = ['chess', 'heads', 'fire', 'office', 'pumpkin', 'redkitchen', 'stairs']
target_dir = '../data/7Scenes_256/'

# Dataset sequence split, key: dataset, value: (train seq, test seq)
split = {'chess' : (['01', '02', '04', '06'],['03', '05']),
         'heads' : (['02'],['01']),  # Reversed 
         'fire' : (['01', '02'], ['03', '04']), 
         'office' : (['01', '03', '04', '05', '08', '10'],['02', '06', '07', '09']), 
         'pumpkin' : (['02', '03', '06', '08'],['01', '07']),
         'redkitchen' : (['01', '02', '05', '07', '08', '11', '13'],['03', '04', '06', '12', '14']), 
         'stairs' : (['02', '03', '05', '06'],['01', '04'])
        }

"""
Format explanation:
In dataset/frame*.pose.txt, pose=[R|t] is camera-to-world which fulfils X_world = R*X_cam + t
In our formulation we want pose_=[R_|c_] which fulfils X_cam = R_*(X_world - c_)
Therefore, each line in the *.new.txt follows: imframe x y z w p q r,
Notice xyz is c_ with c_=t and wpqr is mat2quat(R_) with R_ = R.T, where [R|t] are the pose in the original frame*.pose.txt
"""
train_txt = 'dataset_train.txt'
test_txt = 'dataset_test.txt'
for dataset in datasets:
    print('Generating train and test label text for {} ...'.format(dataset))
    train_num = 0
    test_num = 0
    train_seqs = split[dataset][0]
    with open(os.path.join(target_dir, dataset, train_txt), 'w') as ftrain:
        ftrain.write('7Scenes {} Dataset -- Train Split\n'.format(dataset))
        ftrain.write('ImageFile, Camera Position [X Y Z W P Q R]\n')
        ftrain.write('\n')
        for seq in train_seqs:
            im_regx = os.path.join(origin_dir, dataset,'seq-{}'.format(seq), '*.color.png')
            im_files = glob.glob(im_regx)
            for im_file in im_files:
                frame = 'seq-{}/{}'.format(seq, im_file.split('/')[-1])
                pose_txt = im_file.replace('color.png', 'pose.txt')
                pose = np.loadtxt(pose_txt)
                R = pose[0:3, 0:3]
                q = mat2quat(R.T) # IMPORTANT!
                c = pose[0:3, 3] 
                cur = [frame]
                cur += ["{:.8f}".format(i) for i in c]
                cur += ["{:.8f}".format(i) for i in q]
                ftrain.write(' '.join(cur)+'\n')
                train_num += 1
    print('Save {}, training frame number: {}'.format(ftrain.name, train_num))
    
    test_seqs = split[dataset][1]
    with open(os.path.join(target_dir, dataset, test_txt), 'w') as ftest:
        ftest.write('7Scenes {} Dataset -- Test Split\n'.format(dataset))
        ftest.write('ImageFile, Camera Position [X Y Z W P Q R]\n')
        ftest.write('\n')
        for seq in test_seqs:
            im_regx = os.path.join(origin_dir, dataset,'seq-{}'.format(seq), '*.color.png')
            im_files = glob.glob(im_regx)
            for im_file in im_files:
                frame = 'seq-{}/{}'.format(seq, im_file.split('/')[-1])            
                pose_txt = im_file.replace('color.png', 'pose.txt')
                pose = np.loadtxt(pose_txt)
                R = pose[0:3, 0:3]
                q = mat2quat(R.T)
                c = pose[0:3, 3]
                cur = [frame]
                cur += ["{:.8f}".format(i) for i in c]
                cur += ["{:.8f}".format(i) for i in q]
                ftest.write(' '.join(cur)+'\n')
                test_num += 1
    print('Save {}, testing frame number: {}\n'.format(ftest.name, test_num))

Generating train and test label text for chess ...
Save ../data/7Scenes/chess/dataset_train.txt, training frame number: 4000
Save ../data/7Scenes/chess/dataset_test.txt, testing frame number: 2000

Generating train and test label text for heads ...
Save ../data/7Scenes/heads/dataset_train.txt, training frame number: 1000
Save ../data/7Scenes/heads/dataset_test.txt, testing frame number: 1000

Generating train and test label text for fire ...
Save ../data/7Scenes/fire/dataset_train.txt, training frame number: 2000
Save ../data/7Scenes/fire/dataset_test.txt, testing frame number: 2000

Generating train and test label text for office ...
Save ../data/7Scenes/office/dataset_train.txt, training frame number: 6000
Save ../data/7Scenes/office/dataset_test.txt, testing frame number: 4000

Generating train and test label text for pumpkin ...
Save ../data/7Scenes/pumpkin/dataset_train.txt, training frame number: 4000
Save ../data/7Scenes/pumpkin/dataset_test.txt, testing frame number: 2000

Gene

## Split validation set from train

In [6]:
import random
import os
random.seed(0)
base_dir = '../data/7Scenes_256/'
datasets = ['chess', 'heads', 'fire', 'office', 'pumpkin', 'redkitchen', 'stairs']
val_step = 4

for dataset in datasets:
    counts = {}
    seq_lines = {}
    with open(os.path.join(base_dir, dataset, 'dataset_train.txt'), 'r') as f:
        lines = sorted(f.readlines())
        for line in lines:
            if not line.startswith('seq'):
                continue
            seq = line.split()[0].split('/')[0]
            if seq not in counts:
                counts[seq] = 0
                seq_lines[seq] = []
            counts[seq] += 1
            seq_lines[seq].append(line)
    print('>>>>{} {}'.format(dataset,counts))

    val_lines = []
    train_lines = []
    for seq in counts:
        num = counts[seq]
        for i,line in enumerate(seq_lines[seq]):
            if i % val_step == 0 and i > 0:
                val_lines.append(line)
            else:
                train_lines.append(line)
    print('Val step {} Train: {} Val: {}'.format(val_step, len(train_lines), len(val_lines)))

    train = open(os.path.join(base_dir, dataset, 'train.exp2.txt'), 'w')
    train.writelines(train_lines)
    val = open(os.path.join(base_dir, dataset, 'val.exp2.txt'), 'w')
    val.writelines(val_lines)
    train.close()
    val.close()

>>>>chess {'seq-01': 1000, 'seq-02': 1000, 'seq-04': 1000, 'seq-06': 1000}
Val step 4 Train: 3004 Val: 996
>>>>heads {'seq-02': 1000}
Val step 4 Train: 751 Val: 249
>>>>fire {'seq-01': 1000, 'seq-02': 1000}
Val step 4 Train: 1502 Val: 498
>>>>office {'seq-01': 1000, 'seq-03': 1000, 'seq-04': 1000, 'seq-05': 1000, 'seq-08': 1000, 'seq-10': 1000}
Val step 4 Train: 4506 Val: 1494
>>>>pumpkin {'seq-02': 1000, 'seq-03': 1000, 'seq-06': 1000, 'seq-08': 1000}
Val step 4 Train: 3004 Val: 996
>>>>redkitchen {'seq-01': 1000, 'seq-02': 1000, 'seq-05': 1000, 'seq-07': 1000, 'seq-08': 1000, 'seq-11': 1000, 'seq-13': 1000}
Val step 4 Train: 5257 Val: 1743
>>>>stairs {'seq-02': 500, 'seq-03': 500, 'seq-05': 500, 'seq-06': 500}
Val step 4 Train: 1504 Val: 496
